In [57]:
from fyers_apiv3 import fyersModel
from config import CLIENT_ID, AUTH_CODE, REDIRECT_URI, RESPONSE_TYPE, STATE, SECRET_KEY, GRANT_TYPE, get_logger

# --- Exchange auth_code for access_token ---
session = fyersModel.SessionModel(
    client_id=CLIENT_ID,
    secret_key=SECRET_KEY,
    redirect_uri=REDIRECT_URI,
    response_type=RESPONSE_TYPE,
    state=STATE,
    grant_type=GRANT_TYPE
)

session.set_token(AUTH_CODE)
response = session.generate_token()

access_token = response.get("access_token")

In [58]:
import pandas as pd
import numpy as np

import langchain
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [59]:
fyers = fyersModel.FyersModel(client_id=CLIENT_ID, token=access_token, is_async=False)

data = {"symbols": "NSE:NIFTY50-INDEX"}
response = fyers.quotes(data)
# print(response)

if "d" in response and len(response["d"]) > 0:
    ltp = response["d"][0]["v"]["lp"]  # 'lp' means Last Price
    print(f"Current price of SBIN: ₹{ltp}")
else:
    print("Error fetching price:", response)


Current price of SBIN: ₹25709.85


True

In [72]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from fyers_apiv3 import fyersModel  # Make sure fyers_api is installed

# -----------------------------
# Load environment variables
# -----------------------------
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(GROQ_API_KEY)


# -----------------------------
# Load saved vector store
# -----------------------------
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
print("Vector store loaded!")

# -----------------------------
# Initialize ChatGroq LLM
# -----------------------------
llm = ChatGroq(
    api_key="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",
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

# -----------------------------
# Load company symbol mapping CSV
# -----------------------------
company_df = pd.read_csv("symbols.csv")  # 2400 entries
company_dict = dict(zip(company_df["Company Name"].str.upper(), company_df["Symbol"]))

# -----------------------------
# Initialize Fyers model
# -----------------------------
fyers = fyersModel.FyersModel(client_id=CLIENT_ID, token=access_token, is_async=False)

# -----------------------------
# Define LangChain ChatPromptTemplate
# -----------------------------
system_template = """You are a helpful finance assistant.
You can optionally call a tool to retrieve company stock symbols and prices if needed.
Use the tool results only if relevant, otherwise answer normally."""
system_msg_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = """User Question:
{user_query}

Tool Output (use only if relevant):
{retrieved_info}

Answer:"""
human_msg_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_msg_prompt, human_msg_prompt])

# -----------------------------
# Define tools
# -----------------------------
def symbol_lookup_tool(query_str, top_k=2):
    """
    Returns top_k most similar company entries from the vector store.
    """
    docs = vectorstore.similarity_search(query_str, k=top_k)
    results = [doc.page_content for doc in docs]
    return results

def get_current_price(symbol: str):
    """
    Fetch current price from Fyers API
    """
    data = {"symbols": symbol}
    response = fyers.quotes(data)
    if "d" in response and len(response["d"]) > 0:
        ltp = response["d"][0]["v"]["lp"]  # 'lp' = Last Price
        return ltp
    else:
        return None

# -----------------------------
# Chat loop
# -----------------------------
print("Finance ChatBot ready! Type 'exit' to quit.")

while True:
    user_query = input("You: ")
    print("YOU : ",user_query)
    if user_query.lower() in ["exit", "quit"]:
        break

    # Step 1: Ask LLM if we need to call the symbol lookup tool
    tool_decision_prompt = f"""
You are a chatbot. The user asked: "{user_query}".
Decide if you need to call the 'symbol_lookup_tool' to find company symbols.
If yes, generate a short string to query the tool (do not just repeat the user input).
If no, just respond 'NO_TOOL'.
Return only one line: either 'NO_TOOL' or the query string to pass to the tool.
"""
    decision_response = llm.invoke([
        ("system", "You are a helpful assistant."),
        ("human", tool_decision_prompt)
    ])
    tool_query = decision_response.content.strip()
    
    # Step 2: Call the tool if needed
    if tool_query != "NO_TOOL":
        tool_results = symbol_lookup_tool(user_query)
        if tool_results:
            # Extract SYMBOL using simple split or regex
            import re
            match = re.search(r"SYMBOL:\s*([\w:-]+)", tool_results[0])
            if match:
                symbol = match.group(1)
                # Now call Fyers API with symbol
                data = {"symbols": symbol}
                response = fyers.quotes(data)
                if "d" in response and len(response["d"]) > 0:
                    ltp = response["d"][0]["v"]["lp"]
                    print(f"Current price of {symbol}: ₹{ltp}")
                else:
                    print("Error fetching price:", response)

    else:
        retrieved_info = "No tool call needed."
        print(f"[DEBUG] Model chose NOT to call the tool.")

    # Step 3: Format messages using ChatPromptTemplate
    messages = chat_prompt.format_messages(user_query=user_query, retrieved_info=retrieved_info)

    # Step 4: Get final response from model
    response = llm.invoke(messages)
    
    print("Bot:", response.content)


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
Vector store loaded!
Finance ChatBot ready! Type 'exit' to quit.
YOU :  who has higher prices TCS of tata motor limited
Bot: To determine which company has higher prices between TCS (Tata Consultancy Services) and Tata Motors Limited, we need to consider their stock prices. Generally, TCS, being a leading IT services company, tends to have a higher stock price compared to Tata Motors Limited, which is an automotive manufacturing company. However, stock prices can fluctuate frequently based on market conditions, company performance, and other factors.

Without current, real-time data, it's challenging to provide an exact comparison. But historically, TCS has often had higher stock prices than Tata Motors Limited due to its consistent financial performance and the growth prospects of the IT sector. For the most accurate and up-to-date comparison, I recommend checking the current stock prices of both companies on a financial news we

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key="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",
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

response = llm.invoke([
    ("system", "You are a helpful assistant."),
    ("human", "Say hello in one sentence.")
])

print(response.content)

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(GROQ_API_KEY)


# -----------------------------
# Load saved vector store
# -----------------------------
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
print("Vector store loaded!")

# -----------------------------
# Initialize ChatGroq LLM
# -----------------------------
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)


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


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}